In [1]:
# Importaciones de la biblioteca estándar
import re

# Importaciones de bibliotecas de terceros relacionadas
import nltk
import pandas as pd
import plotly.express as px
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
# Cargar el archivo CSV
df = pd.read_csv('../youtoxic_english_1000.csv')

### 1. Consolidar las etiquetas en una sola columna binaria.

Para el propósito de crear un modelo de aprendizaje automático que reconozca los mensajes de odio, podemos consolidar las múltiples etiquetas de toxicidad en una sola etiqueta binaria. Por ejemplo, si cualquier columna de etiqueta tiene True, podríamos considerar ese comentario como un mensaje de odio.

In [3]:
# Consolidar las etiquetas en una sola columna binaria 'IsHate'
# Consideramos un comentario como mensaje de odio si alguna de las etiquetas relevantes es verdadera
columns_to_consider = ['IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 'IsObscene', 
                       'IsHatespeech', 'IsRacist', 'IsNationalist', 'IsSexist', 
                       'IsHomophobic', 'IsReligiousHate', 'IsRadicalism']

# Cualquier comentario que tenga al menos una etiqueta positiva se considerará de odio
df['IsHate'] = df[columns_to_consider].any(axis=1)

# Ahora seleccionamos solo las columnas necesarias, es decir, 'Text' y 'IsHate'
data_simplified = df[['Text', 'IsHate']]

# Verificamos la distribución de las clases
distribution = data_simplified['IsHate'].value_counts()

data_simplified.head(), distribution


(                                                Text  IsHate
 0  If only people would just take a step back and...   False
 1  Law enforcement is not trained to shoot to app...    True
 2  \nDont you reckon them 'black lives matter' ba...    True
 3  There are a very large number of people who do...   False
 4  The Arab dude is absolutely right, he should h...   False,
 IsHate
 False    538
 True     462
 Name: count, dtype: int64)

- El conjunto de datos se ha simplificado para incluir solo el texto del comentario y una etiqueta binaria IsHate.

- La distribución de las clases es bastante equilibrada, con 538 comentarios etiquetados como no odio (False) y 462 etiquetados como mensajes de odio (True).

### 2. Preprocesamiento del texto de los comentarios.

Este proceso generalmente incluye:

- Convertir todo el texto a minúsculas para mantener la consistencia.

- Eliminar caracteres especiales y números, que suelen ser irrelevantes para el análisis de sentimiento.

- Eliminar las llamadas "stop words" (palabras comunes que no aportan significado al contexto).

- Tokenizar el texto (dividir el texto en palabras o frases individuales).

- Lematizar o realizar stemming (reducir las palabras a su raíz básica).

In [6]:
# Cargar las stopwords una sola vez
stop_words = set(stopwords.words('english'))

# Función para el preprocesamiento del texto
def preprocess_text(text, stop_words):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres no alfabéticos
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenización
    words = word_tokenize(text)
    # Eliminar stopwords
    words = [word for word in words if word not in stop_words]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # Unir las palabras en una cadena de texto de nuevo
    text = ' '.join(words)
    return text

# Aplicar el preprocesamiento a cada comentario de forma segura
data_simplified.loc[:, 'Text'] = data_simplified['Text'].apply(lambda x: preprocess_text(x, stop_words))

# Muestra las primeras filas para verificar
print(data_simplified.head())

                                                Text  IsHate
0  people would take step back make case wasnt an...   False
1  law enforcement trained shoot apprehend traine...    True
2  dont reckon black life matter banner held whit...    True
3  large number people like police officer called...   False
4  arab dude absolutely right shot extra time sho...   False


- La tabla que se muestra al final es el resultado de aplicar tu función de preprocesamiento preprocess_text al DataFrame data_simplified. Muestra las primeras filas con el texto ya preprocesado y la columna IsHate que indica si el comentario es considerado de odio o no.

- Esto significa que tu función de preprocesamiento está trabajando como se esperaba: está limpiando y tokenizando el texto, eliminando las stopwords y aplicando la lematización. 

### 3. Vectorizacion y division del conjunto de datos

Vamos a:

- Vectorizar el texto utilizando TF-IDF.
- Dividir el conjunto de datos en conjuntos de entrenamiento y prueba.

In [7]:
# Vectorización del texto con TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(data_simplified['Text'])

# Convertir la columna 'IsHate' a numérica
y = data_simplified['IsHate'].astype(int)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Verificar las dimensiones de los conjuntos de datos resultantes
(X_train.shape, X_test.shape), (y_train.shape, y_test.shape)

(((800, 4114), (200, 4114)), ((800,), (200,)))

La vectorización con TF-IDF se ha completado y hemos dividido el conjunto de datos en conjuntos de entrenamiento y prueba. El conjunto de entrenamiento tiene 800 ejemplos, y el conjunto de prueba tiene 200 ejemplos. Cada ejemplo está representado por 4646 características únicas obtenidas del TF-IDF.

### 4. Entrenamiento de modelos 

In [ ]:
# Inicializar los modelos con configuraciones predeterminadas
models = {
    'SVM': SVC(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'LightGBM': LGBMClassifier(random_state=42, verbose=-1),
    'XGBoost': XGBClassifier(random_state=42)
}

# DataFrame para almacenar los resultados de los modelos
model_results = pd.DataFrame(columns=['Model', 'Train Accuracy', 'Test Accuracy', 'Overfitting', 'Train Report', 'Test Report', 'Confusion Matrix'])

# Función para entrenar y evaluar modelos
def train_evaluate(models, X_train, y_train, X_test, y_test):
    for name, model in models.items():
        # Entrenar el modelo
        model.fit(X_train, y_train)
        # Predecir en el conjunto de entrenamiento y prueba
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        # Calcular la precisión
        accuracy_train = accuracy_score(y_train, y_pred_train)
        accuracy_test = accuracy_score(y_test, y_pred_test)
        # Calcular el sobreajuste
        overfitting = accuracy_train - accuracy_test
        # Almacenar resultados en el DataFrame
        model_results.loc[len(model_results)] = {
            'Model': name,
            'Train Accuracy': accuracy_train,
            'Test Accuracy': accuracy_test,
            'Overfitting': overfitting,
            'Train Report': classification_report(y_train, y_pred_train),
            'Test Report': classification_report(y_test, y_pred_test),
            'Confusion Matrix': confusion_matrix(y_test, y_pred_test)
        }

# Llamar a la función con los modelos y datos
train_evaluate(models, X_train, y_train, X_test, y_test)

# Configurar Pandas para mostrar todas las filas y columnas en el DataFrame (ajustar si es necesario)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Mostrar los resultados
print(model_results[['Model', 'Train Accuracy', 'Test Accuracy', 'Overfitting']].to_string(index=False))


            Model  Train Accuracy  Test Accuracy  Overfitting
              SVM            1.00          0.680        0.320
    Random Forest            1.00          0.680        0.320
Gradient Boosting            0.86          0.655        0.205
         LightGBM            0.88          0.715        0.165
          XGBoost            0.95          0.740        0.210


In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix

# Configuración para la validación cruzada
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# DataFrames para almacenar los resultados
confusion_matrices = {}
cross_val_results = {}

# Realizar validación cruzada y calcular la matriz de confusión
for name, model in models.items():
    # Validación cruzada
    cv_scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    cross_val_results[name] = cv_scores
    
    # Entrenar y predecir para calcular la matriz de confusión
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    conf_matrix = confusion_matrix(y_test, y_pred)
    confusion_matrices[name] = conf_matrix

# Mostrar resultados
print("Resultados de la Validación Cruzada:\n")
for name, scores in cross_val_results.items():
    print(f"{name}: {scores.mean():.2f} (+/- {scores.std():.2f})")

print("\nMatrices de Confusión:\n")
for name, matrix in confusion_matrices.items():
    print(f"{name}:\n{matrix}\n")


Resultados de la Validación Cruzada:

SVM: 0.66 (+/- 0.03)
Random Forest: 0.69 (+/- 0.03)
Gradient Boosting: 0.70 (+/- 0.05)
LightGBM: 0.67 (+/- 0.02)
XGBoost: 0.68 (+/- 0.02)

Matrices de Confusión:

SVM:
[[86  7]
 [57 50]]

Random Forest:
[[79 14]
 [50 57]]

Gradient Boosting:
[[82 11]
 [58 49]]

LightGBM:
[[77 16]
 [41 66]]

XGBoost:
[[78 15]
 [37 70]]



Los resultados de la validación cruzada te proporcionan dos piezas de información clave sobre cada modelo de clasificación:

- Precisión Media: Es el valor promedio de la precisión (accuracy) que el modelo logra en los diferentes conjuntos de datos creados durante la validación cruzada. La precisión es la proporción de predicciones correctas (tanto positivas como negativas) entre todas las predicciones realizadas. Un valor más alto es mejor.

- Desviación Estándar de la Precisión: Es una medida de cuánto varía la precisión del modelo en los diferentes conjuntos de datos de la validación cruzada. Una desviación estándar baja significa que el modelo tiene un rendimiento más consistente y es menos sensible a las diferencias en los conjuntos de datos específicos utilizados para entrenamiento y prueba.

Para cada modelo, tienes un valor de precisión media seguido de un signo ± y un valor de desviación estándar. 

Aquí está el desglose:

- SVM: Tiene una precisión media del 66% con una variabilidad de 0.03 puntos porcentuales. Esto significa que, en promedio, el modelo clasifica correctamente el 66% de las muestras, y esta precisión varía en un rango de 3% hacia arriba o hacia abajo en diferentes pruebas.

- Random Forest: Tiene una precisión media del 69% con una variabilidad del 3%. Esto indica que es ligeramente más preciso que el SVM en promedio, con la misma cantidad de variabilidad en su rendimiento.

- Gradient Boosting: Tiene la precisión media más alta con 70%, pero con una variabilidad del 5%, lo que sugiere que su rendimiento podría cambiar más entre diferentes pruebas en comparación con los modelos SVM y Random Forest.

- LightGBM: Tiene una precisión media del 67% con una variabilidad del 2%, lo que indica un rendimiento ligeramente mejor que el SVM pero menor que el Random Forest y Gradient Boosting, aunque con un rendimiento muy consistente.

- XGBoost: Muestra una precisión media del 68% con una variabilidad del 2%, lo que significa que es más preciso que el SVM y el LightGBM pero menos que el Random Forest y el Gradient Boosting, con un rendimiento bastante estable a través de diferentes pruebas.